In [3]:
import openai
import pandas as pd
import string
import os
from opencc import OpenCC

pd.set_option('max_columns', 500)
pd.set_option('max_rows', 10000)
pd.set_option('display.max_colwidth', -1)
openai.api_key = 'api-key'

C:\Users\A0258\anaconda3\envs\env_py3.7.6\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  if __name__ == "__main__":


In [180]:
cc = OpenCC('s2twp')

df_raw = pd.read_csv(r"\interview\qa.txt",encoding='utf-8-sig',sep="\t",header=None,names=["prompt","completion"])

df_raw.prompt = df_raw.prompt.apply(cc.convert)
df_raw.completion = df_raw.completion.apply(cc.convert)
df_raw

,prompt,completion
0,飛機上遺失東西怎麼找回？,"三、旅客來電反映將物品遺失在飛機上,若是剛剛下機,可聯絡候機尋找。"
1,憑哪些證件可以到機場派出所辦理臨時登機證明？,黃花機場機公安辦理臨時乘機證明的位置:T二航站樓安檢十六號通道邊設有辦理櫃檯旅客由於特殊原因...
2,航空意外險的購買,購買航空意外險可以透過三種方式：機票搭售、機場購買和網上購買。機票搭售購買的話，通常價格便宜...
3,如何辦理臨時的乘機手續?,黃花機場機公安辦理臨時乘機證明的位置:T二航站樓安檢十六號通道邊設有辦理櫃檯
4,長沙黃花機場航站樓是第幾航站樓,長沙黃花機場T1航站樓，2018年5月16日剛恢復啟用；T1和T2 分別位於磁浮機場站的兩邊...
...,...,...
829,請問***航空公司在幾號航站樓？,T1航站樓：春秋航空9C、九元航空AQ T2航站樓：海南航空HU、首都航空J...
830,請問機場擺渡車乘車地點在哪裡？,機場免費擺渡車乘車地點在T2航站樓隔離區內17號登機口旁和T3航站樓隔離區內T3中轉中心旁。...
831,請問機場可以過夜停車嗎？,西安咸陽國際機場停車場為旅客提供各類大型、小型車輛以及計程車的停車區域，主要包括：普通停車場...
832,請問我開車去機場對路線不熟，在哪裡看地圖？,在西安咸陽國際機場官網的“旅客須知”裡您可以查詢到機場交通路線圖，詳情可撥打諮詢電話：029...


In [181]:
#saving the new csv file
df_raw.to_csv('QA.csv')

#converting csv to jsonl format because that is the format GPT3 can be finetuned in 
openai tools fine_tunes.prepare_data -f "QA資料集.csv"

In [7]:
with open(r'interview\QA資料集_prepared.jsonl', encoding="utf-8") as f:
  response = openai.File.create(file=f, purpose='fine-tune')
  print(response)

{
  "bytes": 86290,
  "created_at": 1678288200,
  "filename": "file",
  "id": "file-O0kRnigIjkIGCIlrETLAfhP2",
  "object": "file",
  "purpose": "fine-tune",
  "status": "uploaded",
  "status_details": null
}


In [8]:
response = openai.FineTune.create(training_file="file-O0kRnigIjkIGCIlrETLAfhP2", model='curie')
print(response)

{
  "created_at": 1678288209,
  "events": [
    {
      "created_at": 1678288209,
      "level": "info",
      "message": "Created fine-tune: ft-byXUwxJLLLY2kQ29NHNV8YDl",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": null,
    "n_epochs": 4,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-byXUwxJLLLY2kQ29NHNV8YDl",
  "model": "curie",
  "object": "fine-tune",
  "organization_id": "org-kNb9OnT0uHVBVAtWEO7dNbg3",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 86290,
      "created_at": 1678288200,
      "filename": "file",
      "id": "file-O0kRnigIjkIGCIlrETLAfhP2",
      "object": "file",
      "purpose": "fine-tune",
      "status": "processed",
      "status_details": null
    }
  ],
  "updated_at": 1678288209,
  "validation_files": []
}


In [11]:
#this shows how much training has been done
response = openai.FineTune.retrieve(id="ft-byXUwxJLLLY2kQ29NHNV8YDl")
print(response)

{
  "created_at": 1678288209,
  "events": [
    {
      "created_at": 1678288209,
      "level": "info",
      "message": "Created fine-tune: ft-byXUwxJLLLY2kQ29NHNV8YDl",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1678288603,
      "level": "info",
      "message": "Fine-tune costs $0.72",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1678288604,
      "level": "info",
      "message": "Fine-tune enqueued. Queue number: 0",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1678288605,
      "level": "info",
      "message": "Fine-tune started",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1678288688,
      "level": "info",
      "message": "Completed epoch 1/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1678288713,
      "level": "info",
      "message": "Completed epoch 2/4",
      "object": "fine-tune-event"
    },
    {
      "created_at": 1678288737,
      "level": "info",


In [42]:
#aviation
#Checking the inference
ft_model = "curie:ft-hugh-2023-03-08-11-00-36"
start_sequence = "\nA:"
restart_sequence = "\n\nQ: "
res = openai.Completion.create(model=ft_model, prompt="\n\nQ:一般商務中心的規格@start_sequence",
                                temperature=1,
                                max_tokens=300,
                                top_p=1,
                                frequency_penalty=0,
                                presence_penalty=0,
                                stop=["\n"])
res["choices"][0]["text"]

'：20間專用室内乘車房（最多40人）機場大廳和大廳設施 ；'

In [7]:
#business
#Checking the inference
ft_model = "curie:ft-hugh-2023-03-08-15-19-37"
start_sequence = "\nA:"
restart_sequence = "\n\nQ: "
res = openai.Completion.create(model=ft_model, prompt="\n\nQ:請問商務中心有免費的課程活動嗎？ @start_sequence",
                                temperature=0.9,
                                max_tokens=300,
                                top_p=1,
                                frequency_penalty=0,
                                presence_penalty=0,
                                stop=["\n"])
res["choices"][0]["text"]

'_point：免費課程一般不會太多，尤其房產開發，公共機構和地段設計，商務中心會幫忙。但有時候商務中心會幫你做為你公司裝潢，但不管是商務中心或是公司裝潢公司秘招都要拿有效票券，才會叫你們全部免費！ END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END END'

In [11]:
df = pd.read_csv('gpt_generate.csv')
df

,Question
0,商務中心是什麼？
1,商務中心缺點
2,商務中心辦公優點
3,誰適合承租商務中心？
4,共享辦公室是什麼?
5,商務中心優點
6,誰適合租借商務中心？
7,商務中心公司登記好嗎？
8,借址登記常見問題總整理
9,商務中心/聯合辦公室能提供什麼樣的服務？


In [ ]:
A=[]

ft_model = "curie:ft-hugh-2023-03-08-15-19-37"

start_sequence = "\nA:"
restart_sequence = "\n\nQ: "
for i in df.Question:
    res = openai.Completion.create(model=ft_model, prompt="\n\nQ:"+i+"@start_sequence",
                                    temperature=1,
                                    max_tokens=300,
                                    top_p=1,
                                    frequency_penalty=0,
                                    presence_penalty=0,
                                    stop=["\n"])
    A.append(res["choices"][0]["text"])

df['Answer'] = A
df